In [1]:
#enabling arrow just for a backup if pandas is needed at any point of time
import numpy as np
import pandas as pd
from datetime import datetime

startTime = datetime.now()
from pyspark.sql import SparkSession
data_directory_path = 'C:/Users/saidh/Downloads/Market Basket Analysis/'

aisles = pd.read_csv(data_directory_path + 'aisles.csv')

departments = pd.read_csv(data_directory_path + 'departments.csv')

order_products_prior = pd.read_csv(data_directory_path + 'order_products__prior.csv')

order_products_train = pd.read_csv(data_directory_path + 'order_products__train.csv')

orders = pd.read_csv(data_directory_path + 'orders.csv')
# Replacing numbers with their corresponding hour representation 

products = pd.read_csv(data_directory_path + 'products.csv') 

In [2]:
order_products = pd.concat([order_products_train, order_products_prior])
order_products = order_products.merge(products, on='product_id', how='left').merge(orders, on='order_id', how='left').merge(departments, on='department_id').merge(aisles, on='aisle_id')

df=order_products
df = df[df['eval_set'] == 'train']
df

In [22]:
product_counts = df.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[0:100].reset_index(drop=True)
product_counts 


,product_id,frequency
0,24852,18726
1,13176,15480
2,21137,10894
3,21903,9784
4,47626,8135
...,...,...
95,8193,1418
96,9387,1379
97,37687,1362
98,20995,1361


In [23]:
freq_products = list(product_counts.product_id)
del product_counts 
freq_products[1:10]


[13176, 21137, 21903, 47626, 47766, 47209, 16797, 26209, 27966]

In [24]:
order_products = df[df.product_id.isin(freq_products)]
del df
order_products.shape
print(order_products)


          order_id  product_id  add_to_cart_order  reordered  \
1828410          1       22035                  8          1   
1828429       2445       22035                  4          0   
1828430       2445       33731                  7          0   
1828431       2445       40604                 20          1   
1828440       2937       40604                 16          1   
...            ...         ...                ...        ...   
29573237   3415183        5077                 11          1   
29573248   3417341        5077                  3          1   
29573251   3417506        5077                 11          0   
29573278   3420258        5077                  4          1   
29573279   3420286        5077                 26          1   

                         product_name  aisle_id  department_id  user_id  \
1828410   Organic Whole String Cheese        21             16   112108   
1828429   Organic Whole String Cheese        21             16      816   
182843

In [25]:
df=order_products[['order_id','product_name','reordered']].set_index('order_id')
df

,product_name,reordered
order_id,,
1,Organic Whole String Cheese,1
2445,Organic Whole String Cheese,0
2445,Grated Parmesan,0
2445,Feta Cheese Crumbles,1
2937,Feta Cheese Crumbles,1
...,...,...
3415183,100% Whole Wheat Bread,1
3417341,100% Whole Wheat Bread,1
3417506,100% Whole Wheat Bread,0


In [26]:
df['product_name'] = df['product_name'].str.strip()


In [27]:
basket = df.groupby(['order_id', 'product_name'])['reordered'].sum().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Blueberries,Boneless Skinless Chicken Breasts,Broccoli Crown,Bunched Cilantro,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [28]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head()

basket_sets1=basket_sets[:100000]

In [29]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = fpgrowth(basket_sets1, min_support=0.001, use_colnames=True)


C:\ProgramData\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [30]:
from pandas import DataFrame
from sqlalchemy import asc


rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

df=pd.DataFrame(rules)

df.head(10).sort_values(by=['antecedent_len'],ascending=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3678,"(Organic Hass Avocado, Organic Kiwi)",(Bag of Organic Bananas),0.002792,0.142376,0.001566,0.561069,3.940750,0.001169,1.953891,2
3728,"(Organic Raspberries, Organic Navel Orange)",(Bag of Organic Bananas),0.001886,0.142376,0.001055,0.559322,3.928482,0.000786,1.946146,2
300,"(Organic Hass Avocado, Organic Raspberries)",(Bag of Organic Bananas),0.008119,0.142376,0.004454,0.548556,3.852868,0.003298,1.899737,2
3722,"(Organic Hass Avocado, Organic Navel Orange)",(Bag of Organic Bananas),0.002664,0.142376,0.001460,0.548000,3.848960,0.001081,1.897398,2
3102,"(Organic Fuji Apple, Strawberries)",(Banana),0.001918,0.176420,0.001044,0.544444,3.086073,0.000706,1.807859,2
3734,"(Organic Navel Orange, Organic Strawberries)",(Bag of Organic Bananas),0.002653,0.142376,0.001428,0.538153,3.779795,0.001050,1.856942,2
44,"(Organic Whole String Cheese, Organic Hass Avo...",(Bag of Organic Bananas),0.002419,0.142376,0.001279,0.528634,3.712942,0.000934,1.819445,2
313,"(Organic Hass Avocado, Organic Raspberries, Or...",(Bag of Organic Bananas),0.002866,0.142376,0.001811,0.631970,4.438737,0.001403,2.330311,3
2493,"(Organic Hass Avocado, Organic Cucumber, Organ...",(Bag of Organic Bananas),0.001811,0.142376,0.001087,0.600000,4.214189,0.000829,2.144060,3
1337,"(Organic Baby Spinach, Organic Strawberries, O...",(Bag of Organic Bananas),0.002515,0.142376,0.001385,0.550847,3.868959,0.001027,1.909427,3


In [31]:
antecedent_len_count = df.groupby('antecedent_len')['antecedents'].count().reset_index().rename(columns = {'antecedent_len':'antecedent_len'})
antecedent_len_count = antecedent_len_count.sort_values('antecedent_len', ascending=True)
antecedent_len_count

,antecedent_len,antecedents
0,1,3068
1,2,870
2,3,12


In [32]:
df=pd.DataFrame(rules)
df.head(10).sort_values(by=['antecedent_len','confidence'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
313,"(Organic Hass Avocado, Organic Raspberries, Or...",(Bag of Organic Bananas),0.002866,0.142376,0.001811,0.631970,4.438737,0.001403,2.330311,3
2493,"(Organic Hass Avocado, Organic Cucumber, Organ...",(Bag of Organic Bananas),0.001811,0.142376,0.001087,0.600000,4.214189,0.000829,2.144060,3
1337,"(Organic Baby Spinach, Organic Strawberries, O...",(Bag of Organic Bananas),0.002515,0.142376,0.001385,0.550847,3.868959,0.001027,1.909427,3
3678,"(Organic Hass Avocado, Organic Kiwi)",(Bag of Organic Bananas),0.002792,0.142376,0.001566,0.561069,3.940750,0.001169,1.953891,2
3728,"(Organic Raspberries, Organic Navel Orange)",(Bag of Organic Bananas),0.001886,0.142376,0.001055,0.559322,3.928482,0.000786,1.946146,2
300,"(Organic Hass Avocado, Organic Raspberries)",(Bag of Organic Bananas),0.008119,0.142376,0.004454,0.548556,3.852868,0.003298,1.899737,2
3722,"(Organic Hass Avocado, Organic Navel Orange)",(Bag of Organic Bananas),0.002664,0.142376,0.001460,0.548000,3.848960,0.001081,1.897398,2
3102,"(Organic Fuji Apple, Strawberries)",(Banana),0.001918,0.176420,0.001044,0.544444,3.086073,0.000706,1.807859,2
3734,"(Organic Navel Orange, Organic Strawberries)",(Bag of Organic Bananas),0.002653,0.142376,0.001428,0.538153,3.779795,0.001050,1.856942,2
44,"(Organic Whole String Cheese, Organic Hass Avo...",(Bag of Organic Bananas),0.002419,0.142376,0.001279,0.528634,3.712942,0.000934,1.819445,2


In [33]:
#df[(df['confidence']>0.5) & (df['antecedent_len']==2) & (df['consequents']!={'Bag of Organic Bananas'})].sort_values(by=['confidence'],ascending=False)

In [34]:
from datetime import datetime

rules[(rules['lift'] >= 1.4) & (rules['confidence'] >= 0.3)].sort_values(by=['confidence', 'lift'], ascending=False).to_excel(r'C:/Users/saidh/Downloads/marketbasketanalysisusingfpgrowth.xlsx', index=False)
print('It took ', datetime.now() - startTime, ' to run')

It took  0:02:29.216014  to run


In [35]:
tt1=datetime.now()-startTime
print('It took ', tt1, ' to run')

It took  0:02:29.303074  to run


In [37]:

import pandas as pd
from matplotlib import pyplot as plt

fig = plt.figure(figsize =(10, 7))
plt.bar(tt,tt1)
plt.show()

NameError: name 'tt' is not defined

<Figure size 720x504 with 0 Axes>

In [36]:

# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName(
 #   'Read CSV File into DataFrame').getOrCreate()


    
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")


#Top 5 orders in the orders dataframe
# orders.show(n=5)

# products.show(n=5)

# order_products_train.show(n=5)

# order_products_prior.show(n=5)

# orders.show(n=5)

# departments.show(n=5)

# aisles.show(n=5)

# from pyspark.sql.functions import *
# from pyspark import SparkContext
# from pyspark.sql import SQLContext
# from pyspark.sql.types import *
# from pyspark.sql.functions import *

# from pyspark.sql.functions import sum,avg,max,min,mean,count
# from pyspark import SparkContext
# from pyspark.sql import SQLContext


# df = spark.sql("select count(order_id) as total_orders, order_hour_of_day as hour from orders group by order_hour_of_day order by order_hour_of_day")
# df.show()

# df2=spark.sql("select count(order_id) as total_orders,(case when order_dow = '0' then 'Sunday'when order_dow = '1' then 'Monday' when order_dow = '2' then 'Tuesday' when order_dow = '3' then 'Wednesday' when order_dow = '4' then 'Thursday' when order_dow = '5' then 'Friday' when order_dow = '6' then 'Saturday' end) as day_of_week from orders group by order_dow order by total_orders desc")
# df.show()
